In [40]:
import random

class Dog:

    def __init__(self, name):
        self.name = name
        self.tricks = []    # creates a new empty list for each dog

    def add(self, num):
        for i in range(num):
            print(i)
            x = self.hello(i)
        return x
    def hello(self,i):
        return 2**i

o=Dog('Fido')
o.add(5)

0
1
2
3
4


16